## 기본 세팅

### LangSmith 설정
- [랭스미스](https://smith.langchain.com/o/f7b9cff6-45e5-5a17-ac66-babbd820380c/)
- 반응 추적

#### Ref
- [테디노트](https://wikidocs.net/250954)

In [10]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "PDF Chatbot with Chroma & groq "
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGSMITH_API_KEY")

GROQ_API_KEY = os.getenv("GROQ_API_KEY")

### Loader
- 호환성을 위해서 DirectoryLoader 기본으로 사용
- pdf자료가 하나가 아닌, 여러 형태이기때문에 UnstructuredPDFLoader 고려

In [14]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import UnstructuredPDFLoader

pdf_folder_path = "/home/jun/my_project/langchain_tutorial/data"

%time
PyPDF_loader = DirectoryLoader(pdf_folder_path, glob="**/*.pdf", loader_cls=PyPDFLoader,show_progress=True)
PyPDF_documents = PyPDF_loader.load()

# %time # pip install unstructured 설치필요하다...
# UnstructuredPDF_loader = DirectoryLoader(pdf_folder_path, glob="**/*.pdf", loader_cls=UnstructuredPDFLoader,show_progress=True)
# UnstructuredPDF_documents = UnstructuredPDF_loader.load()

CPU times: user 1 μs, sys: 0 ns, total: 1 μs
Wall time: 4.53 μs


100%|██████████| 1/1 [00:01<00:00,  1.73s/it]


In [21]:
print(len(PyPDF_documents))
print(PyPDF_documents[3]) # 차후에 그래프나 그림도 인식하는 멀티모달이 목표

34
page_content='글로벌 Top 6 빅테크 기업 AI 투자 동향
2Part 1 Top 6 빅테크 전체 AI 투자 현황 
Top 6 빅테크 R&D 분석
  ○ 선정된 6대 빅테크 R&D 합산 규모는 ’15년 510억 달러에서 ’23년 4배 이상 증가한 2,387 억 달러로 성장, 
매출에서 차지하는 비중 또한 ’15년 10%에서 ’23년 14%로 증가해 AI를 포함한 IT시장을 선도하기 위해서 
R&D의 역할이 중요해지고 있음을 방증
    - 빅테크의 R&D 연간 규모(’19~’23 년 평균)로 보면 Amazon 이 연간 587억 달러로 가장 높은 수준이며 , 
Google(340 억 달러), Meta(261 억 달러), Microsoft(245 억 달러) 순으로 나타남
    - 빅테크의 R&D 연간 지출 규모(’19~’21 년 평균)는 주요국의 1년 정부 과학기술 R&D 예산보다 큰 수준
       ※ ’19~21 년 평균 R&D 예산: Amazon(44.9$b) >  Google(28.4$b) > 일본(26.3$b) > 영국(17.5$b) > 캐나다 (10.6$b)
[그래프 1] 빅테크의 R&D 규모 비교
*자료원: 각사 SEC 자료 정리(2024), OECD(2024), EU(2024)(‘19-23 평균)' metadata={'source': '/home/jun/my_project/langchain_tutorial/data/글로벌 Top 6 빅테크 기업 AI 투자 동향.pdf', 'page': 3}


### Splitter

In [23]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(PyPDF_documents)

print(len(split_docs))

58


In [25]:
for i,split_doc in enumerate(split_docs[0:3]):
    print(f"{i}번째 문서\n\n{split_doc}")

0번째 문서

page_content='2024글로벌 Top 6 빅테크 기업 AI 투자 동향' metadata={'source': '/home/jun/my_project/langchain_tutorial/data/글로벌 Top 6 빅테크 기업 AI 투자 동향.pdf', 'page': 0}
1번째 문서

page_content='「The AI Report 」는 인공지능 기술‧산업‧정책의 글로벌 이슈와 동향, 시사점을 적시에 분석, 인공지능 
현안에 빠르게 대응하고 관련 정책을 지원하기 위해 한국지능정보사회진흥원 (NIA) 에서 기획‧발간하고 
있습니다 .
1.본 보고서는 방송통신발전기금으로 수행하는 정보통신 ·방송 연구개발 사업의 결과물이므로 , 보고서 내용을  
발표할 때는 반드시 과학기술정보통신부 정보통신 ·방송 연구개발 사업의 연구 결과임을 밝혀야 합니다 .
2.한국지능정보사회진흥원 (NIA)의 승인 없이 본 보고서의 무단전재를 금하며 , 가공·인용할 때는 반드시 
출처를 「한국지능정보사회 진흥원 (NIA) 」이라고 밝혀 주시기 바랍니다 .
3. 본 보고서의 내용은 한국지능정보사회진흥원 (NIA) 의 공식 견해와 다를 수 있습니다 .
▶ 발행인 : 황 종 성
▶ 작 성
   - 한국지능정보사회진흥원 인공지능정책본부 미래전략팀
     김소미 선임연구원 (somikim@nia.or.kr )' metadata={'source': '/home/jun/my_project/langchain_tutorial/data/글로벌 Top 6 빅테크 기업 AI 투자 동향.pdf', 'page': 1}
2번째 문서

page_content='글로벌 Top 6 빅테크 기업 AI 투자 동향 
NIA 미래전략팀 김소미 선임연구원 (somikim@nia.or.kr) THE AI REPORT 2024-2 | 2024. 9. 9.
1 
분석 대상 선정 기준  
  ○ AI 투자를 주도하고 있는 기업을 분석하기 위해 분석 대상은 △전세계 Tech 기업 시가총액 상위 20개1) 중, 
△인공

### Embedding model
- 범용적으로 많이 사용하는 bge사용

In [26]:
%%time
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-m3"
model_kwargs = {"device":"cpu"}
encode_kwargs = {"normalize_embeddings":True}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs,encode_kwargs=encode_kwargs
)

/home/jun/.local/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/jun/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


CPU times: user 4.37 s, sys: 408 ms, total: 4.78 s
Wall time: 6.86 s


# Chroma
- 제일 중요한 부분

### Ref
- [랭체인 공식 문서 : only chroma](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
- [Chroma에 데이터 동적 저장](https://stackoverflow.com/questions/76650513/dynamically-add-more-embedding-of-new-document-in-chroma-db-langchain)
- [랭체인 공식 문서 : rag chroma](https://python.langchain.com/docs/how_to/vectorstores/)
- [테디노트 깃허브](https://github.com/teddylee777/langchain-kr/blob/main/09-VectorStore/01-Chroma.ipynb)

## from_documents

In [44]:
from langchain_chroma import Chroma

vector_store2 = Chroma.from_documents(
    documents=split_docs,
    embedding=embeddings,
    collection_name="chroma_db",
    persist_directory="./chroma_test2_db"
)

In [45]:
vector_store2.similarity_search("google") # 잘 된 모습, 처음 한 번만 해야함

[Document(metadata={'page': 20, 'source': '/home/jun/my_project/langchain_tutorial/data/글로벌 Top 6 빅테크 기업 AI 투자 동향.pdf'}, page_content='($m)투자\n조직국적 대상 기업 설명\n지\n분\n투\n자Anthrophic\n(생성AI)2023년\n(3회)2,30018,400\n35)Google미국⋅OpenAI\x00출신이\x00창업한\x00생성AI\x00Claude336)\x00개발사로\x00\n생성AI\x00역량\x00강화\x00목적\n⋅Amazon도\x002023.9월과\x002024.3월에\x00거쳐\x0040억\x00\n달러\x00투자\x00발표\nSambaNova\n(하드웨어 )‘18-’21년\n(4회)1654,425GV37)미국⋅자체\x00제조\x00AI\x00칩과\x00함께\x00기업용\x00AI\x00플랫폼\x00제공하며,\x00\nGV의\x00AI\x00하드웨어\x00투자로선\x00첫\x00번째\x00사례\nGlance\n(콘텐츠 )2020년145855Google인도⋅인도의\x00스마트폰\x00잠금화면에\x00개인화된\x00콘텐츠와\x00숏폼\x00\n영상서비스\x00Roposo를\x00제공하는\x00AI기반\x00SW\x00회사로\x00\n인도\x00내\x001.2억\x00명\x00사용자\x00확보\nLightmatter\n(하드웨어 )‘19-’23년\n(3회)821,045GV미국⋅저전력\x00데이터센터용\x00칩으로\x00전기\x00대신\x00빛을\x00사용\n하는\x00컴퓨팅\x00제품을\x00개발하는\x00스타트업\nDatabricks\n(데이터분석 )2021년\n2023년7242,500CaptialG미국⋅클라우드\x00기반\x00기업용\x00AI\x00데이터\x00분석\x00플랫폼\n⋅2023년\x0016억\x00달러\x00매출(YoY\x00+50%)\x00기록하며,\x00AI와\x00\n데이터\x00관리\x00기술에\x00집중\x00투자

In [55]:
# 저장된 데이터 확인
vector_store2.get()

{'ids': ['007dde22-ba5e-4c3e-8f59-d33ee5b739ac',
  '009f6266-8b36-4e3d-ab28-cebbb52f10d0',
  '0a4e4ba4-6c4b-4a16-9c49-28e97df31bba',
  '0beddde1-457c-43fe-850b-926d71d9de93',
  '0e615b71-60d6-41f6-a1f5-c3f37e7707e6',
  '182933c0-0a01-41b2-bafa-1cffeaa3ce57',
  '1d068ce6-92d1-4074-a7af-d003f93d7c18',
  '1e13149e-1a6c-4a68-a7b4-d7b565850127',
  '210d50c0-27b6-436b-a7a7-25e395a222d5',
  '323b4b33-37a0-4ed0-b07c-f920cf1f9224',
  '32405b80-2a47-4200-9860-8233b6e2ff72',
  '32686d5f-7c63-45ac-94d5-0617fcd1d62a',
  '32bc89a3-948a-4216-b698-e5127e1288bb',
  '49d395c7-6fc7-428e-9366-e70233d35be4',
  '4a3c7fc5-f35e-4d06-8393-c961e7289926',
  '4b7d7af3-b2c1-41ee-8949-b35c6c37b9f6',
  '4d87b359-e0e2-4a42-96bf-fa5dfabf0428',
  '50d7c1c7-466d-411e-94a0-db8eed4ceeec',
  '51c53fe7-d809-4391-9c2a-8de3ca7e5852',
  '56c663ad-1007-4a4b-a8e3-33ee6db9c5bd',
  '59733c70-a53c-4219-a689-d3ac9271b21a',
  '63ba0d4a-8e63-407c-af95-7b455f1e411b',
  '715847ba-b208-4ad0-9fe1-3ce9871a671d',
  '823ca82a-aa5a-44fb-a151-

In [57]:
DB_PATH = "./chroma_test2_db"
# 디스크에서 문서를 로드합니다.
persist_db = Chroma(
    persist_directory=DB_PATH,
    embedding_function=embeddings,
    collection_name="my_db",
)

In [58]:
persist_db.get() # collection_name이게 기존 것이랑 다르면 아무것도 못가져옴

{'ids': [],
 'embeddings': None,
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents']}

In [59]:
DB_PATH = "./chroma_test2_db"
# 디스크에서 문서를 로드합니다.
persist_db = Chroma(
    persist_directory=DB_PATH,
    embedding_function=embeddings,
    collection_name="chroma_db",
)

In [62]:
len(persist_db.get()['ids']) # 기존것 잘 가져온모습!
# splitter로 나눠진 문서가 총 58개인데, 벡터화된것도 58개로 딱 맞음

58

## only chorma + add_documents

In [29]:
from langchain_chroma import Chroma

# 이건 그냥 만들기만 한거네??
vector_store = Chroma(
    collection_name="onlyone_pdf",
    embedding_function=embeddings,
    persist_directory="./chroma_test_db",  # Where to save data locally, remove if not necessary
)

In [28]:
vector_store

In [30]:
vector_store.similarity_search("Top 6 빅테크 알려줘") # 문서 정보를 전혀 받지 못한 모습

[]

In [33]:
def add_pdf_to_chroma(pdf_path, vectorstore):
    # PDF 로드
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    
    # 텍스트 분할
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(documents)
    
    # ChromaDB에 추가
    vectorstore.add_documents(splits)

    print(f"Added {pdf_path} to ChromaDB")

In [37]:
import os

pdf_folder = "/home/jun/my_project/langchain_tutorial/data"
for filename in os.listdir(pdf_folder):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(pdf_folder, filename)
        add_pdf_to_chroma(pdf_path, vector_store)

Added /home/jun/my_project/langchain_tutorial/data/글로벌 Top 6 빅테크 기업 AI 투자 동향.pdf to ChromaDB


In [38]:
vector_store.similarity_search("goog e") # 추가된 모습, 중복을 피할수가 없구나... 추가한만큼 다 중복되서 벡터화된다

[Document(metadata={'page': 20, 'source': '/home/jun/my_project/langchain_tutorial/data/글로벌 Top 6 빅테크 기업 AI 투자 동향.pdf'}, page_content='($m)투자\n조직국적 대상 기업 설명\n지\n분\n투\n자Anthrophic\n(생성AI)2023년\n(3회)2,30018,400\n35)Google미국⋅OpenAI\x00출신이\x00창업한\x00생성AI\x00Claude336)\x00개발사로\x00\n생성AI\x00역량\x00강화\x00목적\n⋅Amazon도\x002023.9월과\x002024.3월에\x00거쳐\x0040억\x00\n달러\x00투자\x00발표\nSambaNova\n(하드웨어 )‘18-’21년\n(4회)1654,425GV37)미국⋅자체\x00제조\x00AI\x00칩과\x00함께\x00기업용\x00AI\x00플랫폼\x00제공하며,\x00\nGV의\x00AI\x00하드웨어\x00투자로선\x00첫\x00번째\x00사례\nGlance\n(콘텐츠 )2020년145855Google인도⋅인도의\x00스마트폰\x00잠금화면에\x00개인화된\x00콘텐츠와\x00숏폼\x00\n영상서비스\x00Roposo를\x00제공하는\x00AI기반\x00SW\x00회사로\x00\n인도\x00내\x001.2억\x00명\x00사용자\x00확보\nLightmatter\n(하드웨어 )‘19-’23년\n(3회)821,045GV미국⋅저전력\x00데이터센터용\x00칩으로\x00전기\x00대신\x00빛을\x00사용\n하는\x00컴퓨팅\x00제품을\x00개발하는\x00스타트업\nDatabricks\n(데이터분석 )2021년\n2023년7242,500CaptialG미국⋅클라우드\x00기반\x00기업용\x00AI\x00데이터\x00분석\x00플랫폼\n⋅2023년\x0016억\x00달러\x00매출(YoY\x00+50%)\x00기록하며,\x00AI와\x00\n데이터\x00관리\x00기술에\x00집중\x00투자

## 기존 파일과 같은 폴더에 있으면서, 새로운 문서만 추가하는 법!..
너무 코드가 비효율적으로 밖에 안짜진다.. 그냥 수동으로 추가하는 게 더 낫나?

In [ ]:
import os
import hashlib
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import time

def get_file_hash(file_path):
    with open(file_path, "rb") as f:
        file_hash = hashlib.md5()
        chunk = f.read(8192)
        while chunk:
            file_hash.update(chunk)
            chunk = f.read(8192)
    return file_hash.hexdigest()

def process_new_pdfs(folder_path, db):
    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            file_path = os.path.join(folder_path, filename)
            file_hash = get_file_hash(file_path)
            
            # 중복 확인
            if not db.get(where={"hash": file_hash}):
                loader = PyPDFLoader(file_path)
                pages = loader.load_and_split()
                
                text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
                texts = text_splitter.split_documents(pages)
                
                embeddings = OpenAIEmbeddings()
                db.add_documents(texts, metadatas=[{"hash": file_hash} for _ in texts])
                print(f"Added {filename} to the database.")

def main():
    folder_path = "path/to/your/pdf/folder"
    db = Chroma(persist_directory="path/to/chromadb")

if __name__ == "__main__":
    main()